# ДОМАШНЕЕ ЗАДАНИЕ 4. Рекомендательные системы и Spark MLlib

**Дисциплина:** Методы обработки больших данных  
**Студент:** Попов Я.Ю. 
**Группа:** ИУ6-31М

## Решение для малой выборки

### Задание 1. Анализ датасета

1. Выведите данные, сопоставляющие жанры и количество фильмов
2. Выведите первые 10 фильмов с наибольшим количеством рейтингов для каждого жанра в соответствии с вариантом
3. Выведите первые 10 фильмов с наименьшим количеством рейтингов (но больше 10) для каждого жанра в соответствии с вариантом
4. Выведите первые 10 фильмов с наибольшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра в соответствии с вариантом
5. Выведите первые 10 фильмов с наименьшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра в соответствии с вариантом


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, count, avg, row_number
from pyspark.sql.window import Window

# Инициализация SparkSession
spark = SparkSession.builder \
    .appName("MovieRecommendationAnalysis") \
    .getOrCreate()

# Загрузка данных
movies_df = spark.read.csv("movies.csv", header=True, inferSchema=True)
ratings_df = spark.read.csv("ratings.csv", header=True, inferSchema=True)

# Список требуемых жанров
genres = [
    "Animation", "Romance", "Documentary", "Drama", 
    "Comedy", "Musical", "Thriller", "Sci-Fi", "Adventure"
]

/home/ubuntu/ML/anaconda3/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


## Задача 1: Количество фильмов по жанрам

In [3]:
# Разделение на жанры по "|"
movies_exploded = movies_df.withColumn("genre", explode(split(col("genres"), "\\|")))
genre_counts = (
    movies_exploded
    .filter(col("genre").isin(genres))
    .groupBy("genre")
    .count()
    .orderBy(col("count").desc())
)

print("Задача 1: Количество фильмов по жанрам")
genre_counts.show()

Задача 1: Количество фильмов по жанрам
+-----------+-----+
|      genre|count|
+-----------+-----+
|      Drama| 4361|
|     Comedy| 3756|
|   Thriller| 1894|
|    Romance| 1596|
|  Adventure| 1263|
|     Sci-Fi|  980|
|  Animation|  611|
|Documentary|  440|
|    Musical|  334|
+-----------+-----+



## Задача 2: Топ-10 фильмов с наибольшим количеством рейтингов для каждого жанра

In [5]:
# Агрегация рейтингов
movie_ratings = (
    ratings_df
    .groupBy("movieId")
    .agg(
        count("rating").alias("rating_count"),
        avg("rating").alias("avg_rating")
    )
)

# Объединение с информацией о фильмах
movies_enriched = (
    movies_df
    .join(movie_ratings, "movieId", "inner")
    .withColumn("genre", explode(split(col("genres"), "\\|")))
    .filter(col("genre").isin(genres))
)


window_desc = Window.partitionBy("genre").orderBy(col("rating_count").desc())
top_by_count = (
    movies_enriched
    .withColumn("rank", row_number().over(window_desc))
    .filter(col("rank") <= 10)
    .select("genre", "title", "rating_count", "rank")
    .orderBy("genre", "rank")
)

print("\nЗадача 2: Топ-10 по количеству рейтингов")
top_by_count.show(n=100, truncate=False)


Задача 2: Топ-10 по количеству рейтингов
+-----------+------------------------------------------------------------------------------+------------+----+
|genre      |title                                                                         |rating_count|rank|
+-----------+------------------------------------------------------------------------------+------------+----+
|Adventure  |Star Wars: Episode IV - A New Hope (1977)                                     |251         |1   |
|Adventure  |Jurassic Park (1993)                                                          |238         |2   |
|Adventure  |Toy Story (1995)                                                              |215         |3   |
|Adventure  |Star Wars: Episode V - The Empire Strikes Back (1980)                         |211         |4   |
|Adventure  |Independence Day (a.k.a. ID4) (1996)                                          |202         |5   |
|Adventure  |Apollo 13 (1995)                                         

## Задача 3: Топ-10 фильмов с наименьшим количеством рейтингов (>10) для каждого жанра

In [6]:

window_asc = Window.partitionBy("genre").orderBy(col("rating_count").asc())
filtered_movies = movies_enriched.filter(col("rating_count") > 10)
bottom_by_count = (
    filtered_movies
    .withColumn("rank", row_number().over(window_asc))
    .filter(col("rank") <= 10)
    .select("genre", "title", "rating_count", "rank")
    .orderBy("genre", "rank")
)
print("\nЗадача 3: Наименьшее количество рейтингов (>10)")
bottom_by_count.show(truncate=False)


Задача 3: Наименьшее количество рейтингов (>10)
+---------+--------------------------------------+------------+----+
|genre    |title                                 |rating_count|rank|
+---------+--------------------------------------+------------+----+
|Adventure|Hidalgo (2004)                        |11          |1   |
|Adventure|Young Sherlock Holmes (1985)          |11          |2   |
|Adventure|Wild Bunch, The (1969)                |11          |3   |
|Adventure|Bulletproof Monk (2003)               |11          |4   |
|Adventure|All Dogs Go to Heaven 2 (1996)        |11          |5   |
|Adventure|Edge, The (1997)                      |11          |6   |
|Adventure|Oliver & Company (1988)               |11          |7   |
|Adventure|Medicine Man (1992)                   |11          |8   |
|Adventure|Avengers, The (1998)                  |11          |9   |
|Adventure|Chitty Chitty Bang Bang (1968)        |11          |10  |
|Animation|How the Grinch Stole Christmas! (1966)|11  

## Задача 4: Топ-10 фильмов с наибольшим средним рейтингом (>10 рейтингов)

In [7]:
window_avg_desc = Window.partitionBy("genre").orderBy(
    col("avg_rating").desc(),
    col("rating_count").desc()
)
top_by_avg = (
    filtered_movies
    .withColumn("rank", row_number().over(window_avg_desc))
    .filter(col("rank") <= 10)
    .select("genre", "title", "avg_rating", "rating_count", "rank")
    .orderBy("genre", "rank")
)
print("\nЗадача 4: Наивысший средний рейтинг (>10)")
top_by_avg.show(truncate=False)


Задача 4: Наивысший средний рейтинг (>10)
+---------+------------------------------------------------------------------------------+------------------+------------+----+
|genre    |title                                                                         |avg_rating        |rating_count|rank|
+---------+------------------------------------------------------------------------------+------------------+------------+----+
|Adventure|Lawrence of Arabia (1962)                                                     |4.3               |45          |1   |
|Adventure|Outlaw Josey Wales, The (1976)                                                |4.25              |18          |2   |
|Adventure|Princess Bride, The (1987)                                                    |4.232394366197183 |142         |3   |
|Adventure|Star Wars: Episode IV - A New Hope (1977)                                     |4.231075697211155 |251         |4   |
|Adventure|Yojimbo (1961)                                    

## Задача 5: Топ-10 фильмов с наименьшим средним рейтингом (>10 рейтингов)

In [8]:
window_avg_asc = Window.partitionBy("genre").orderBy(
    col("avg_rating").asc(),
    col("rating_count").desc()
)
bottom_by_avg = (
    filtered_movies
    .withColumn("rank", row_number().over(window_avg_asc))
    .filter(col("rank") <= 10)
    .select("genre", "title", "avg_rating", "rating_count", "rank")
    .orderBy("genre", "rank")
)
print("\nЗадача 5: Наименьший средний рейтинг (>10)")
bottom_by_avg.show(n=100, truncate=False)


Задача 5: Наименьший средний рейтинг (>10)
+-----------+---------------------------------------------------------------------------+------------------+------------+----+
|genre      |title                                                                      |avg_rating        |rating_count|rank|
+-----------+---------------------------------------------------------------------------+------------------+------------+----+
|Adventure  |Superman IV: The Quest for Peace (1987)                                    |1.6875            |16          |1   |
|Adventure  |Karate Kid, Part III, The (1989)                                           |1.75              |14          |2   |
|Adventure  |Dungeons & Dragons (2000)                                                  |1.8333333333333333|12          |3   |
|Adventure  |Rambo III (1988)                                                           |1.9166666666666667|12          |4   |
|Adventure  |Anaconda (1997)                                       

# Остановка SparkSession

In [9]:
spark.stop()

### Задание 2. Коллаборативная фильтрация

Вариант 1. По схожести пользователей

Вариант 2. По схожести объектов

1. Разделите данные с рейтингами на обучающее (train_init - 0.8) и тестовое подмножества (test - 0.2), определите среднее значение рейтинга в обучающем подмножестве и вычислите rmse для тестового подмножества, если для всех значений из test предсказывается среднее значение рейтинга
2. Реализуйте коллаборативную фильтрацию в соответствии с вариантом. Для определения схожести используйте train_init, для расчета rmse - test
3. Определите rmse для тестового подмножества


In [10]:
surname = "Попов" 

alp = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
w = [1, 42, 21, 21, 34,  6, 44, 26, 18, 44, 38, 26, 14, 43,  4, 49, 45,
        7, 42, 29,  4,  9, 36, 34, 31, 29,  5, 30,  4, 19, 28, 25, 33]

d = dict(zip(alp, w))
variant =  sum([d[el] for el in surname.lower()]) % 40 + 1

print("Задача № 2: ", variant % 2 + 1 )

Задача № 2:  1


## Инициализация SparkSession

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, broadcast, avg, lit, sqrt
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.window import Window
import time

spark = SparkSession.builder \
    .appName("CollaborativeFiltering") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

ratings_df = spark.read.csv("ratings.csv", header=True, inferSchema=True)

/home/ubuntu/ML/anaconda3/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


## 1: Разделение данных и вычисление RMSE для среднего значения

In [12]:
# Разделение данных
train_init, test = ratings_df.randomSplit([0.8, 0.2], seed=42)

# Вычисление среднего рейтинга в обучающей выборке
avg_rating = train_init.agg(avg("rating").alias("avg_rating")).collect()[0]["avg_rating"]
print(f"Средний рейтинг в обучающей выборке: {avg_rating:.4f}")

test_with_pred = test.withColumn("prediction", lit(avg_rating))

# Расчет RMSE вручную
rmse_base = test_with_pred.withColumn("squared_error", (col("rating") - col("prediction"))**2)
rmse_base = rmse_base.agg(sqrt(avg("squared_error")).alias("rmse")).collect()[0]["rmse"]
print(f"RMSE базовой модели (предсказание средним): {rmse_base:.4f}")

Средний рейтинг в обучающей выборке: 3.5039
RMSE базовой модели (предсказание средним): 1.0504


## 2: Реализация коллаборативной фильтрации по схожести пользователей

In [13]:
# Подготовка данных для ALS
als_train = train_init.select(
    col("userId").cast("integer").alias("user"),
    col("movieId").cast("integer").alias("item"),
    col("rating").cast("float")
)

als_test = test.select(
    col("userId").cast("integer").alias("user"),
    col("movieId").cast("integer").alias("item"),
    col("rating").cast("float")
)

# Обучение ALS модели
als = ALS(
    maxIter=5,
    regParam=0.01,
    userCol="user",
    itemCol="item",
    ratingCol="rating",
    coldStartStrategy="drop",
    seed=42
)

start_time = time.time()
model = als.fit(als_train)
print(f"Модель обучена за {time.time() - start_time:.2f} секунд")

predictions = model.transform(als_test)

Модель обучена за 6.84 секунд


## 3: Определение RMSE для тестового подмножества

In [14]:
# Расчет RMSE с использованием RegressionEvaluator
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

rmse_als = evaluator.evaluate(predictions)
print(f"RMSE ALS модели: {rmse_als:.4f}")

# Альтернативный расчет RMSE вручную
rmse_manual = predictions.withColumn("squared_error", (col("rating") - col("prediction"))**2)
rmse_manual = rmse_manual.agg(sqrt(avg("squared_error")).alias("rmse")).collect()[0]["rmse"]
print(f"RMSE (ручной расчет): {rmse_manual:.4f}")

# Сравнение результатов
print("\nСравнение моделей:")
print(f"Базовая модель (среднее): RMSE = {rmse_base:.4f}")
print(f"ALS модель:              RMSE = {rmse_als:.4f}")
print(f"Улучшение:               {rmse_base - rmse_als:.4f}")

# Дополнительная информация о модели
print("\nХарактеристики модели:")
print(f"Число пользователей в обучении: {model.userFactors.count()}")
print(f"Число фильмов в обучении:      {model.itemFactors.count()}")
print(f"Размерность векторов:          {len(model.userFactors.first()['features'])}")

RMSE ALS модели: 1.0943
RMSE (ручной расчет): 1.0943

Сравнение моделей:
Базовая модель (среднее): RMSE = 1.0504
ALS модель:              RMSE = 1.0943
Улучшение:               -0.0438

Характеристики модели:
Число пользователей в обучении: 610
Число фильмов в обучении:      8954
Размерность векторов:          10


## Остановка SparkSession

In [15]:
spark.stop()